# TDT4136: Introduction to AI
## Assignment 2: Applying the A* search
## 2. A* Implementation

We start by implement an A* algorithm

In [ ]:
import Map as m
import numpy as np
import heapq

class Map(m.Map_Obj):
    """
    The map object improved to implement A* algorithm with utility methods associated to it
    """

    def in_the_map(self, pos):
        return 0 <= pos[0] < self.int_map.shape[0] and 0 <= pos[1] < self.int_map.shape[1]

    def not_walls(self, pos): 
        return self.get_cell_value(pos) > -1

    def neighbors(self, pos):
        (x, y) = pos
        neighbors = [(x+1, y), (x-1, y), (x, y-1), (x, y+1)]
        neighbors.reverse()
        if (x + y) % 2 == 0: neighbors.reverse()
        return filter(self.not_walls, filter(self.in_the_map, neighbors))
    
heuristic_modes = ['manhattan', 'euclidean', 'task5']

def heuristic(a, b, mode: str = 'manhattan') -> float:
    assert mode in heuristic_modes, f"{mode} is not a norm implemented yet"
    (x1, y1) = a
    (x2, y2) = b
    if mode == 'manhattan':
        return abs(x1 - x2) + abs(y1 - y2)
    if mode == 'euclidean':
        return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** (1/2)
    if mode == 'task5':
        return abs(x1 - x2) + abs(y1 -.25 - y2)


class PriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self):
        return heapq.heappop(self.elements)[1]
    
def a_star_algorithm(start, goal, map: Map = Map(task=1), heuristic_mode: str = 'manhattan'):
    assert heuristic_mode in heuristic_modes, f"{heuristic_mode} is not yet an implemented norm"

    start, goal = tuple(start), tuple(goal)

    frontier = PriorityQueue()
    frontier.put(start, 0)

    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
    
        if current == goal:
            while current != start:
                map.replace_map_values(pos=current, value=0, goal_pos=goal)
                current = came_from[current]
            break
        
        for next in map.neighbors(current):
            new_cost = cost_so_far[current] + map.get_cell_value(next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(next, goal, heuristic_mode)
                frontier.put(next, priority)
                came_from[next] = current

    map_int, _ = map.get_maps()
    map.print_map(map_int)
    return cost_so_far[goal]

## 3. Part 1 – Grid with obstacles
### 3.1 Task 1
We have to find the shortest path from Rundhallen (our location) to Strossa using our implementation of the A* algorithm

In [ ]:
map_task1 = Map(task=1)
map_int, _ = map_task1.get_maps()
start, goal = tuple(map_task1.get_start_pos()), tuple(map_task1.get_goal_pos())

path_cost= a_star_algorithm(map=map_task1, start=start, goal=goal)

print(f"The cost of this least-cost path is {path_cost}")
map_task1.show_map()

### 3.2 Task 2

A* algorithm using Manhattan distance for the heuristic function $h$.

In [ ]:
map_task2 = Map(task=2)

start, goal = tuple(map_task2.get_start_pos()), tuple(map_task2.get_goal_pos())
path_cost = a_star_algorithm(map=map_task2, start=start, goal=goal, heuristic_mode='manhattan')

print(f"The cost of this least-cost path is {path_cost}")
map_task2.show_map()

A* algorithm using Euclidean distance for the heuristic function $h$.

In [ ]:
map_task2 = Map(task=2)

start, goal = tuple(map_task2.get_start_pos()), tuple(map_task2.get_goal_pos())
path_cost = a_star_algorithm(map=map_task2, start=start, goal=goal, heuristic_mode='euclidean')

print(f"The cost of this least-cost path is {path_cost}")
map_task2.show_map()

## 4. Part 2 – Grids with different costs
### 4.1 Task 3

In [ ]:
map_task3 = Map(task=3)

start, goal = tuple(map_task3.get_start_pos()), tuple(map_task3.get_goal_pos())
path_cost = a_star_algorithm(map=map_task3, start=start, goal=goal, heuristic_mode='manhattan')

print(f"The cost of this least-cost path is {path_cost}")
map_task3.show_map()

### 4.2 Task 4

In [ ]:
map_task4 = Map(task=4)

start, goal = tuple(map_task4.get_start_pos()), tuple(map_task4.get_goal_pos())
path_cost = a_star_algorithm(map=map_task4, start=start, goal=goal, heuristic_mode='manhattan')

print(f"The cost of this least-cost path is {path_cost}")
map_task4.show_map()

## 5. Part 3 - Moving Goal (Optionnal)

### 5.1 Task 5

In [ ]:
def a_star_moving_goal(start, goal, map: Map = Map(task=1), heuristic_mode: str = 'manhattan'):
    assert heuristic_mode in heuristic_modes, f"{heuristic_mode} is not yet an implemented norm"

    start, goal = tuple(start), tuple(goal)

    frontier = PriorityQueue()
    frontier.put(start, 0)

    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    last_cost = 0

    while not frontier.empty() :
        current = frontier.get()
        if cost_so_far[current] == last_cost + 1: goal = tuple(map.tick())
        last_cost = cost_so_far[current]
        if current == goal:
            while current != start:
                map.replace_map_values(pos=current, value=0, goal_pos=goal)
                current = came_from[current]
            break
        
        for next in map.neighbors(current):
            new_cost = cost_so_far[current] + map.get_cell_value(next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(next, goal, heuristic_mode)
                frontier.put(next, priority)
                came_from[next] = current
        
    return cost_so_far[goal]

map_task5 = Map(task=5)
start, goal = tuple(map_task5.get_start_pos()), tuple(map_task5.get_goal_pos())

path_cost = a_star_moving_goal(map=map_task5, start=start, goal=goal, heuristic_mode='task5')

goal = tuple(map_task5.get_goal_pos())
print(f"The cost of this least-cost path is {path_cost}")
map_task5.show_map()

map_task5 = Map(task=5)
start, goal = tuple(map_task5.get_start_pos()), tuple(map_task5.get_goal_pos())

path_cost = a_star_moving_goal(map=map_task5, start=start, goal=goal, heuristic_mode='manhattan')

goal = tuple(map_task5.get_goal_pos())
print(f"The cost of this least-cost path is {path_cost}")
map_task5.show_map()